In [1]:
pip install -U Flask-SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask-login

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bcrypt

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install flask_admin

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install flask_user

  Using cached Flask-User-1.0.2.2.tar.gz (74 kB)
     |████████████████████████████████| 45 kB 125 kB/s eta 0:00:011
     |████████████████████████████████| 525 kB 508 kB/s eta 0:00:01
     |████████████████████████████████| 111 kB 14.0 MB/s eta 0:00:01
  Created wheel for flask-user: filename=Flask_User-1.0.2.2-py2.py3-none-any.whl size=130481 sha256=aaa9ffe06c919f69dc2411ae24e780282bb8f3a9141dc056a6fd21b22b0a9961
  Stored in directory: /Users/helenm/Library/Caches/pip/wheels/47/78/33/26881ca332f1eb98b10341efefd5c1e1235942f3d417c5f53b
  Created wheel for Flask-Mail: filename=Flask_Mail-0.9.1-py3-none-any.whl size=7568 sha256=541c0e6c71d836856861b6c91890d78384b875cd8a9375fe5fb9b54d6474754b
  Stored in directory: /Users/helenm/Library/Caches/pip/wheels/98/bc/8c/34c329e4d7efeaf7b9886db0c76d0b23170e54de443f688e3c
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13451 sha256=d2b9334ba037c22f50c58c7b140b92a28def7759f77de974094c88c704336c78
  Stored in directory: /User

In [ ]:
from flask import Flask, render_template, url_for, request, redirect, session, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_user import roles_required, UserManager
from datetime import datetime
from flask_bcrypt import Bcrypt
import pandas as pd



app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'abc'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'



class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    notes = db.relationship('Todo', backref='writer', lazy='dynamic')
    roles = db.relationship('Role', secondary='user_roles')
    def is_admin(self):
        if self.id == 1:
            return True
        else:
            return False

        
class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=datetime.utcnow)
    todo_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    complete_time = db.Column(db.String(50),nullable=True)

class Myproject(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=datetime.utcnow)
    complete_time = db.Column(db.String(50),nullable=True)

class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer(), primary_key=True)
    name = db.Column(db.String(50), unique=True)

class UserRoles(db.Model):
    __tablename__ = 'user_roles'
    id = db.Column(db.Integer(), primary_key=True)
    user_id = db.Column(db.Integer(), db.ForeignKey('user.id', ondelete='CASCADE'))
    role_id = db.Column(db.Integer(), db.ForeignKey('roles.id', ondelete='CASCADE'))
       
db.create_all()

if not User.query.filter(User.username == 'adminuser').first():
        user = User(
            username='adminuser',
            email='11@11.com',
            password=bcrypt.generate_password_hash('12345'),
        )
        user.roles.append(Role(name='Admin'))
        user.roles.append(Role(name='Agent'))
        db.session.add(user)
        db.session.commit()
        
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route("/")
def index():
    prices = [{'coffee type': 'Espresso', 'price': '$3' },
              {'coffee type': 'Latte', 'price': '$4' },
              {'coffee type': 'Americano', 'price': '$3' },
              {'coffee type': 'Mocha Latte', 'price': '$5' }]
    
    df = pd.DataFrame(prices)
    df.to_csv('prices.csv',mode='w',index=False)
    df = pd.read_csv("prices.csv")
    list = df.to_dict('records')
    return render_template("index.html", entries = list)

@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        user = User.query.filter_by(username=request.form["username"]).first()
        if user:
            if bcrypt.check_password_hash(user.password, request.form["password"]):
                login_user(user) 
                return redirect(url_for("task"))
 
        return 'Wrong username or password!'
    return render_template("login.html")
    
@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))


@app.route("/products")
def products():
    return render_template("products.html")

@app.route("/store")
def store():
    return render_template("store.html")

@app.route("/about")
def about():
    return render_template("about.html")

@app.route("/users")
def users():
    try:
        users = User.query.order_by(User.id.asc()).all()
        print("Entry successfully retrieved.")
    except Exception as e:
        print("Exception occurred.",e)
    list = []
    for user in users:
        print(user.id, user.username, user.email)
        dict = {'id':user.id, 'username':user.username, 'email':user.email}
        print(dict)
        list.append(dict)
    return render_template('users.html', entries = list)

@app.route("/signup", methods=["GET", "POST"])
def signup():
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        hashed_password = bcrypt.generate_password_hash(request.form['password'])
        print(username,email,hashed_password)
        new_user = User(username=username,email=email,password=hashed_password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect(url_for('login'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('signup.html')


@app.route("/task", methods=["GET", "POST"])
@login_required
def task():
    if request.method == "POST":
        task_content = request.form['content']
        new_task = Todo(content=task_content, todo_writer=session["_user_id"])
        try:
            db.session.add(new_task)
            db.session.commit()
            return redirect(url_for('task')) 
        except Exception as e:
            print("Exception ocurred.",e)
            return 'There was a problem adding your task'
    else:
        tasks = Todo.query.order_by(Todo.date_created).all()
        return render_template('task.html', tasks=tasks)

@app.route('/admintask', methods=["GET", "POST"])
@login_required
def admintask():
    if request.method == "POST" and current_user.is_admin():
        task_content = request.form['content']
        complete_time = request.form['complete_time']
        new_task = Myproject(content=task_content, complete_time=complete_time)
        try:
            db.session.add(new_task)
            db.session.commit()
            return redirect(url_for('admintask')) 
        except Exception as e:
            print("Exception ocurred.",e)
            return 'There was a problem adding your task'
    else:
        tasks = Myproject.query.order_by(Myproject.date_created).all()
        return render_template('admintask.html', tasks=tasks)


@app.route('/delete/<int:id>')
@login_required
def delete(id):
    task_to_delete = Todo.query.get_or_404(id)
    try:
        db.session.delete(task_to_delete)
        db.session.commit()
        return redirect(url_for('task'))
    except:
        return 'There was a problem deleting that task'

@app.route('/update/<int:id>', methods=['GET', 'POST'])
@login_required
def update(id):
    task = Todo.query.get_or_404(id)

    if request.method == 'POST':
        task.content = request.form['content']

        try:
            db.session.commit()
            return redirect(url_for('task'))
        except:
            return 'There was an issue updating your task'

    else:
        return render_template('update.html', task=task)

if __name__ == '__main__':
    app.run('localhost', 9009)

/Users/helenm/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9009/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2021 15:22:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:22:08] "GET /admintask HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 15:22:08] "GET /login?next=%2Fadmintask HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:22:10] "GET /admintask HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 15:22:10] "GET /login?next=%2Fadmintask HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:22:17] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 15:22:17] "GET /task HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:22:25] "GET /admintask HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:33:12] "POST /admintask HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 15:33:12] "GET /admintask HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:34:52] "POST /admintask HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 15:34:52] "GET /admintask HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:35:07] "POST /admintask HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 15

eleanor 123@123.com b'$2b$12$5FgG/nIeJmEQuD4ecYNFqOj6BZpJlKAOo/yeQpYBPTrIYPnTdsgxq'
Registered!


127.0.0.1 - - [01/Nov/2021 15:42:43] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [01/Nov/2021 15:42:43] "GET /task HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:42:49] "GET /admintask HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:42:56] "POST /admintask HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 15:43:02] "POST /admintask HTTP/1.1" 200 -
